In [0]:
import glob
import re
file_annotator1 = glob.glob('/content/drive/My Drive/Skripsi/Data/Annotator 1/*.anns')
file_annotator2 = glob.glob('/content/drive/My Drive/Skripsi/Data/Annotator 2/*.anns')

In [0]:
def raw_text(file):
  raw = []
  for fd in file:
    f = open(fd, 'r')
    raw.append(f.readlines())
  return raw

raw_annotator1 = raw_text(file_annotator1)
raw_annotator2 = raw_text(file_annotator2)

In [0]:
def separate_alnum(s, sentence):
  prev = ""
  for idx_c, c in enumerate(s):
    if c.isalnum():
      prev += c
      if idx_c == len(s)-1:
        sentence.append([prev, "O"])
    else:
      if prev != "":
        sentence.append([prev, "O"])
        prev = ""
      sentence.append([c, "O"])

def preprocess(raw):
  preprocessed = []
  for txt in raw:
    sentence = []
    count_sentence = 0
    prev_token = []
    prev_tag = ""
    for token in txt:
      curr_token = token.split()
      if len(curr_token) > 1:
        curr_word = curr_token[0]
        curr_tag = curr_token[1]

        if curr_tag == "O":
          if prev_tag != "":
            prev_token.append(prev_tag)
            sentence.append(prev_token)
            
            prev_token = []
            prev_tag = ""

          separate_alnum(curr_word, sentence)
            
          if re.search("(\.$)", curr_word):
            preprocessed.append(sentence)
            sentence = []
            count_sentence += 1

        else:
          prev_token.append(curr_word)
          prev_tag = re.sub("[BMES]\-","",curr_tag)

      elif count_sentence == 0:
        if prev_tag != "":
          prev_token.append(prev_tag)
          sentence.append(prev_token)
          prev_token = []
          prev_tag = ""
        preprocessed.append(sentence)
        sentence = []
        count_sentence += 1
  return preprocessed

preprocessed_annotator1 = preprocess(raw_annotator1)
preprocessed_annotator2 = preprocess(raw_annotator2)

In [0]:
def save_preprocessed(directory, preprocessed):
  f = open(directory, "w")
  for sentence in preprocessed:
    for words in sentence:
      f.write(" ".join(words) + "\n")
    f.write("\n")
  f.close()

dir_annotator1 = "/content/drive/My Drive/Skripsi/Data/Annotator 1/preprocessed_annotator1.txt"
save_preprocessed(dir_annotator1, preprocessed_annotator1)

dir_annotator2 = "/content/drive/My Drive/Skripsi/Data/Annotator 2/preprocessed_annotator2.txt"
save_preprocessed(dir_annotator2, preprocessed_annotator2)